In [ ]:
%load_ext autoreload
%autoreload 2
import logging
import warnings
import os
import pandas as pd
from config import imap_password_customplanet, db_username, db_password
from Scraping_Emails.modules.scrapes import scrape
from Scraping_Emails.modules.db_operations_aws import DatabaseConnector
from MySQL_Scripts.modules.email_failures import  gather_and_send_bad_emails_to_db
warnings.filterwarnings("ignore", category=UserWarning, module="pandas")

#create log dir and file
logpath_creation = os.getcwd() + '\\Logs'
if not os.path.exists(logpath_creation):
    os.makedirs(logpath_creation)

logging.basicConfig(filename= logpath_creation + '\\Email_Scraper.log', level=logging.INFO,
                    format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', force=True)

#What is the difference between update_bad_emails & gather and send bad emails to db
#Consolidating update_bad_emails to email_faulures


email_address = 'team@customplanet.com'
email_pass = imap_password_customplanet
server = 'emailcampaign.c9vhoi6ncot7.us-east-1.rds.amazonaws.com'
database = 'emailcampaign'
db = 'emailcampaign'


# ----------------------------------Updating Faulty Emails in the DB before Send----------------------------------
#Searches GMAIL on the daily for bad emails that were sent back, and sends data to unsubscribed email table
#Goes through all distinct subject lines in the DB

db_connector_email_history = DatabaseConnector(server, database, db_username, db_password, db, 'email_history')
query = db_connector_email_history.SQL_query('SELECT DISTINCT subject FROM [emailcampaign].[dbo].[email_history]')
    
#Iterate through all unique subject lines to update the db table with all faulty email addresses
#Keep in mind this assumes that the func is ran everyday with the function filtering up to yesterday
for line in query['subject']:
    print('\n')
    print(f'Prior Unique Subject Line: {line}')
                                    # subject_line, email_address, email_pass, log_type, override_date = None
    gather_and_send_bad_emails_to_db(line, email_address, email_pass, 'Prior Unique Subject Line', '3/4/2024')


#Special chars can not be recognized in subejct lines such as '&'
#For example 'Local Supplier for Baseball Uniforms & Apparel' is not recognized